In [3]:
import tensorflow as tf
import numpy as np
import os 
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = '/tmp/data'
SAVER_DIR = 'saved_model1'
DIR = 'saved_model1'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# Saver 저장 

In [2]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]), name='W')

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
gd_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

saver = tf.train.Saver(max_to_keep=7, keep_checkpoint_every_n_hours=1)   # Saver Define

with tf.Session() as sess:

    # Train
    sess.run(tf.global_variables_initializer())
    for step in range(1,NUM_STEPS+1):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        _, accuracy_out = sess.run([gd_step, accuracy], feed_dict={x: batch_xs, y_true: batch_ys})
        if step % 50 == 0:
            print('step({:5}) Accuracy: {}'.format(step, accuracy_out))
            saver.save(sess, os.path.join(SAVER_DIR, "model_ckpt"), global_step=step)  # Saver 저장

    ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})
    print("Accuracy: {:.4}%".format(ans*100))

print("Complete !!!")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step(   50) Accuracy: 0.7300000190734863
step(  100) Accuracy: 0.7699999809265137
step(  150) Accuracy: 0.7799999713897705
step(  200) Accuracy: 0.7900000214576721
step(  250) Accuracy: 0.8500000238418579
step(  300) Accuracy: 0.8999999761581421
step(  350) Accuracy: 0.8399999737739563
step(  400) Accuracy: 0.7200000286102295
step(  450) Accuracy: 0.8299999833106995
step(  500) Accuracy: 0.8700000047683716
step(  550) Accuracy: 0.8199999928474426
step(  600) Accuracy: 0.8399999737739563
step(  650) Accuracy: 0.8700000047683716
step(  700) Accuracy: 0.8600000143051147
step(  750) Accuracy: 0.8600000143051147
step(  800) Accuracy: 0.9399999976158142
step(  850) Accuracy: 0.7900000214576721
step(  900) Accuracy: 0.8700000047683716
step(  950) Accuracy: 0.8799999952316284
step( 1000) Accuracy: 0.87999

# Restore

In [6]:
tf.reset_default_graph() 
x      = tf.placeholder(tf.float32, [None, 784],name='x')
y_true = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784, 10]),name='W')
y_pred = tf.matmul(x, W)
cross_entropy = tf.reduce_mean(
             tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,labels=y_true))
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    saver.restore(sess, os.path.join(SAVER_DIR,"model_ckpt-1000")) 
    ans, pred, out_w = sess.run([accuracy, y_pred, W], feed_dict={x: data.test.images, y_true: data.test.labels})
    print("테스트 데이터 정확도 (Restored) : %f" % accuracy.eval(feed_dict={x: data.test.images, y_true: data.test.labels}))
    print("Accuracy: {:.4}%".format(ans*100))

INFO:tensorflow:Restoring parameters from saved_model1/model_ckpt-1000
테스트 데이터 정확도 (Restored) : 0.098000
Accuracy: 9.8%


In [7]:
tf.reset_default_graph() 
x = tf.placeholder(tf.float32, [None, 784],name='x')
W = tf.Variable(tf.zeros([784, 10]),name='W')
y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)
cross_entropy = tf.reduce_mean(
             tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,labels=y_true))
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, os.path.join(DIR,"model_ckpt-1000"))
    ans = sess.run(accuracy, feed_dict={x: data.test.images, 
                                        y_true: data.test.labels})

print("Accuracy: {:.4}%".format(ans*100))

INFO:tensorflow:Restoring parameters from saved_model1/model_ckpt-1000
Accuracy: 9.8%


In [4]:
print('true:{}  pred:{}'.format(data.test.labels[0], pred[0]))
print('true:{}  pred:{}'.format(data.test.labels[1], pred[1]))
print('true:{}  pred:{}'.format(data.test.labels[2], pred[2]))
print('true:{}  pred:{}'.format(data.test.labels[3], pred[3]))
print('true:{}  pred:{}'.format(data.test.labels[4], pred[4]))


true:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]  pred:[-0.2997764  -2.396625   -0.55709255  0.14885554 -0.3006323  -0.6809314
 -1.9654436   4.9229593  -0.36305875  1.4917457 ]
true:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]  pred:[ 0.8274702  -0.41580135  3.0696874   1.6521825  -3.380368    1.3781973
  2.1888726  -3.367156    0.85198987 -2.8050747 ]
true:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]  pred:[-1.5278001   3.4892929   0.38342446  0.16641429 -1.1307375  -0.5344057
 -0.2316672  -0.3421754   0.33610243 -0.60844857]
true:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]  pred:[ 4.873728   -4.3641305  -0.02257991 -0.59475064 -2.1310449   1.2108917
  1.7043753   0.5620216  -0.7776005  -0.4609098 ]
true:[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]  pred:[-0.2912239  -3.3865573   0.59476346 -1.2455342   2.8272822  -0.78046924
  0.1962466   0.61305845  0.21885556  1.2535785 ]


In [120]:
saver[0]

TypeError: 'Saver' object does not support indexing